## Libraries

In [1]:
# Importing data from the r/politics
from psaw import PushshiftAPI
from datetime import datetime, timezone, date
import pandas as pd
import os
import re
import numpy as np
from tqdm import tqdm, tqdm_pandas
from nltk.draw.dispersion import dispersion_plot
from collections import Counter
import datetime as dt
from PIL import Image
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
tqdm.pandas()

C:\Users\JaQtae\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
mpl.rcParams.update(mpl.rcParamsDefault)
def setup_mpl():
    #mpl.rcParams['font.family'] = 'Helvetica Neue'
    mpl.rcParams['font.size'] = '14'
    mpl.rcParams['lines.linewidth'] = 3
setup_mpl()

# WordCloud
Loading in the data 

In [4]:
local_storage = local_storage = r"C:\Users\JaQtae\Desktop\SocInfo2022\Data\FINAL_COMMENTS.csv"
com_data = pd.read_csv(local_storage,index_col=0,parse_dates=[0])
com_data.index = com_data.index.rename('dates')
# The collected comments data set still contained some data from 2020-09-30. This is removed. 

# Make sure all text bodies are of type string. 
com_data["body"] = com_data["body"].apply(str)
com_data.head()

,id,link_id,score,author,parent_id,body,parent_author,tokens,politician,children_comments,mentions_Trump,mentions_Biden,compound_sentiment_score
dates,,,,,,,,,,,,,
2020-11-02 23:26:26,gay6kwb,t3_jmybs3,1,J_Class_Ford,t1_gay5x5w,Division is his weapon,cyanydeez,"['division', 'weapon']",Trump,[],NaN,NaN,-0.2960
2020-11-02 23:20:27,gay5x5w,t3_jmybs3,1,cyanydeez,t1_gay5pjk,"dunno, everything trump does seems to generate...",J_Class_Ford,"['dunno', 'everything', 'trump', 'seems', 'gen...",Trump,['gay6kwb'],True,NaN,0.0000
2020-11-02 23:18:33,gay5pjk,t3_jmybs3,1,J_Class_Ford,t1_gay5doq,Not sure it generates many more votes. I work ...,cyanydeez,"['not', 'sure', 'generates', 'many', 'votes', ...",Trump,['gay5x5w'],True,NaN,-0.0160
2020-11-02 23:15:36,gay5doq,t3_jmybs3,1,cyanydeez,t1_gay3si8,"yeah, but for every vote this generates, would...",J_Class_Ford,"['yeah', 'every', 'vote', 'generates', 'would'...",Trump,['gay5pjk'],NaN,NaN,0.5450
2020-11-02 23:04:18,gay43vt,t3_jmybs3,1,Tesides,t3_jmybs3,I really do believe this idiot is trying to lo...,geoxol,"['i', 'really', 'believe', 'idiot', 'trying', ...",Trump,[],NaN,NaN,-0.7632


### Grouping the data by the associated politician and make it into a single corpus

In [5]:
author_bodies = com_data.groupby(["politician"]).apply(lambda x: x["body"].unique())
author_bodies

politician
Biden    [I'm crazy. I'm not evil. I'm just like many o...
Trump    [Division is his weapon, dunno, everything tru...
dtype: object

In [6]:
trump_corpus = " ".join(author_bodies["Trump"])
biden_corpus = " ".join(author_bodies["Biden"])

### Tokenization
Includes removal of stopwords, URLS and normalizes the text by lowercasing all characters.
Note: We could remove numbers?

We apply this to the corpus', and remove webscraping occurences of &gt and &amp for both.

In [7]:
# We solve this by defining the clean_tokens function below.
# Define stop words to also include punctuation
import nltk
stop = set(stopwords.words('english') + list(string.punctuation))

# Function to tokenize and clean the text of each submission
def clean_tokens(text):
    tokens = nltk.word_tokenize(text)
    # In the list comprehension below, we exclude URL's, stopwords and numbers as well as setting all characters to lowercase
    
    clean_tokens = [re.sub(r'http\S+', '', str(i)).lower() for i in tokens if str(i).isalpha()]
    clean_tokens = [t for t in clean_tokens if t not in stop]
    return clean_tokens

TTC = clean_tokens(trump_corpus)
TBC = clean_tokens(biden_corpus)

In [8]:
TTC_j = " ".join(TTC)
TBC_j = " ".join(TBC)
# Hader regex
TTC_j = TTC_j.replace(' gt', '')
TTC_j = TTC_j.replace(' amp', '')
TBC_j = TBC_j.replace(' gt', '')
TBC_j = TBC_j.replace(' amp', '')

In [9]:
# Define stop words to also include punctuation
stop = set(stopwords.words('english') + list(string.punctuation))
# Function to tokenize and clean the text of each submission
def clean_tokens(text):
    tokens = nltk.word_tokenize(text)
    clean_tokens = [re.sub(r'http\S+', '', str(i)).lower() for i in tokens if str(i).isalpha()]
    clean_tokens = [t for t in clean_tokens if t not in stop]
    return clean_tokens

com_data["tokens"] = com_data.progress_apply(lambda x: clean_tokens(x["body"]), axis=1)

100%|██████████████████████████████████████████████████████████████████████████| 91672/91672 [00:28<00:00, 3269.95it/s]


In [10]:
com_data

,id,link_id,score,author,parent_id,body,parent_author,tokens,politician,children_comments,mentions_Trump,mentions_Biden,compound_sentiment_score
dates,,,,,,,,,,,,,
2020-11-02 23:26:26,gay6kwb,t3_jmybs3,1,J_Class_Ford,t1_gay5x5w,Division is his weapon,cyanydeez,"[division, weapon]",Trump,[],NaN,NaN,-0.2960
2020-11-02 23:20:27,gay5x5w,t3_jmybs3,1,cyanydeez,t1_gay5pjk,"dunno, everything trump does seems to generate...",J_Class_Ford,"[dunno, everything, trump, seems, generate, re...",Trump,['gay6kwb'],True,NaN,0.0000
2020-11-02 23:18:33,gay5pjk,t3_jmybs3,1,J_Class_Ford,t1_gay5doq,Not sure it generates many more votes. I work ...,cyanydeez,"[sure, generates, many, votes, work, assumptio...",Trump,['gay5x5w'],True,NaN,-0.0160
2020-11-02 23:15:36,gay5doq,t3_jmybs3,1,cyanydeez,t1_gay3si8,"yeah, but for every vote this generates, would...",J_Class_Ford,"[yeah, every, vote, generates, would, think, c...",Trump,['gay5pjk'],NaN,NaN,0.5450
2020-11-02 23:04:18,gay43vt,t3_jmybs3,1,Tesides,t3_jmybs3,I really do believe this idiot is trying to lo...,geoxol,"[really, believe, idiot, trying, lose, purpose]",Trump,[],NaN,NaN,-0.7632
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-01 01:25:30,g78ozrz,t3_j2vwwt,1,5DollarHitJob,t3_j2vwwt,Can they just appeal this?,Plymouth03,[appeal],Trump,[],NaN,NaN,0.0000
2020-10-01 01:24:25,g78ovcd,t3_j2vwwt,1,AmishTechno,t3_j2vwwt,"And, nothing will happen.",Plymouth03,"[nothing, happen]",Trump,[],NaN,NaN,0.0000
2020-10-01 00:29:42,g78j6ey,t3_j2vwwt,1,ryhaltswhiskey,t1_g78iult,Ah I did not know that\r\n\r\nMan I hope Barr ...,memepolizia,"[ah, know, man, hope, barr, ends, jail]",Trump,[],NaN,NaN,0.4404


In [11]:
com_data.to_csv("FINAL_COMMENTS.csv")

In [ ]:
#TTC_j

In [ ]:
Biden_mask = np.array(Image.open("Illustrations/Biden_silhouette_background.png"))
wcB = WordCloud(background_color="white", 
                      mask=Biden_mask,
                      contour_width=3, 
                      repeat=True,
                      min_font_size=3,
                      contour_color='darkgreen').generate(TBC_j)
plt.figure(figsize=(10,4))
plt.imshow(wcB, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()
#wcB.to_file('WordCloud_Biden.png')

In [ ]:
Trump_mask = np.array(Image.open("Illustrations/Trump_silhouette_background.png"))
wcT = WordCloud(background_color="white", 
                      mask=Trump_mask,
                      contour_width=3, 
                      repeat=True,
                      min_font_size=3,
                      contour_color='darkgreen').generate(TTC_j)
plt.figure(figsize=(10,4))
plt.imshow(wcT, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()
#wcT.to_file('WordCloud_Trump.png')

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
wcT = WordCloud(width=1400, height=800).generate(TTC_j)
plt.figure(figsize=(28,16))
plt.imshow(wcT, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()
wcT.to_file('WordCloud_Trump.png')

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
wcB = WordCloud(width=1400, height=800).generate(TBC_j)
plt.figure(figsize=(28,16))
plt.imshow(wcB, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()
wcB.to_file('WordCloud_Biden.png')

# Validation of dictionary based methods
In order for it to work, the rule-of-thumb is to have at least 10.000 tokens per document. We group them daily over the period and plot it with the desired cutoff. 

In [ ]:
com_data['cleaned_tokens'] = com_data['body'].progress_apply(lambda x: clean_tokens(x))
com_data['date'] = com_data.index
com_data['test'] = com_data['date'].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
documents_per_day = com_data.groupby("test").cleaned_tokens.sum()

In [ ]:
for i in tqdm(range(len(documents_per_day))):
    #Remove gt's and amp's
    c1 = documents_per_day[i].count('gt')
    c2 = documents_per_day[i].count('amp')
    k = max(c1, c2)
    for j in range(0,k):
        try:
            documents_per_day[i].remove('gt')
            documents_per_day[i].remove('amp')
        except Exception:
            pass
        
    #Remove nothings ''
    c3 = documents_per_day[i].count('')
    for j in range(0,c3):
        try:
            documents_per_day[i].remove('')
        except Exception:
            pass


## Log-scaled for better viewablity.

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax = plt.subplots(figsize=(6,3), dpi=400)
#plt.xticks(rotation='vertical')
ax.bar(documents_per_day.index, [len(doc) for doc in documents_per_day], color= "b")
ax.set_title("Daily length of documents")
ax.axhline(10000, color="r", linestyle="dashed", label="Cutoff")
ax.set_ylabel("Number of tokens (log)")
ax.xaxis.set_major_locator(plt.MaxNLocator(7))
plt.xticks(rotation = 25)
ax.set_yscale('log')
ax.legend()
plt.show()

We disregard the last 3 days in the analysis going forward, partially due to them not living up to the criteria for the methods used, but also due to the difference in comments each day for each candidate, ie. Trump has comments on 2020-11-04, while Biden does not, whereas the reverse is true for 2020-11-04.

# Happiness scores using the Hedonometer lexicon.
We load in the Hedonometer data regarding happiness scores for words and apply it in the analysis of the daily documents in the corpus.

In [ ]:
labMT = pd.read_csv(r"C:\Users\JaQtae\Desktop\SocInfo2022\Hedonometer.csv", index_col="Word")
setup_mpl()

In [ ]:
def hscore(tokens, p=False):
    #Iterative counter
    # If we want print p = True.
    score = 0
    freq_dict = dict(nltk.FreqDist(tokens))
    freq_norm = 0
    for word in tokens:
        # Count for word in dict
        try:
            score += labMT.loc[[word]]["Happiness Score"].values[0] * freq_dict[word]
            freq_norm += freq_dict[word]
        except:
            #Doesnt exist in labMT
            None
    hscore = score / (freq_norm+1e-06) #NaN for 0-division
    if p==True:
        return print("Happiness score: {}".format(hscore))
    else:
        return hscore

In [ ]:
temp = com_data.groupby(['test', 'politician']).cleaned_tokens.sum()
for i in tqdm(range(len(temp))):
    #Removal of webscrape nonsense
    c1 = temp[i].count('gt')
    c2 = temp[i].count('amp')
    k = max(c1, c2)
    for j in range(0,k):
        try:
            temp[i].remove('gt')
            temp[i].remove('amp')
        except Exception:
            pass

    c3 = temp[i].count('')
    for j in range(0,c3):
        try:
            temp[i].remove('')
        except Exception:
            pass

In [ ]:
P_daily_hscore = temp.progress_apply(lambda x: hscore(x))
P_daily_hscore.index = temp.index

In [ ]:
# Convert to csv for ease of plotting
#P_daily_hscore.to_csv("P_daily_hscore.csv")

### Here we redefine the dataset to another format that is more maleable, ie. pandas dataframes with the indexing wanted.

In [ ]:
B_hscores = []
T_hscores = []
for q in range(len(P_daily_hscore)):
    if P_daily_hscore.index[q].count('Biden') == 1:
        B_hscores.append(P_daily_hscore[q])
    elif P_daily_hscore.index[q].count('Trump') == 1:
        T_hscores.append(P_daily_hscore[q])
    else:
        pass

In [ ]:
#Sanity check
len(B_hscores) == len(T_hscores)

In [ ]:
#Dataframes are easier to manage in the form from Week7
df_B = pd.DataFrame()
df_T = pd.DataFrame()

In [ ]:
df_B['B_hscore'] = B_hscores
df_T['T_hscore'] = T_hscores

In [ ]:
catcher = []
tester = []
for k in range(len(P_daily_hscore)):
    if P_daily_hscore.index[k].count('Biden') == 1:
        catcher.append(P_daily_hscore.index[k][0])
    elif P_daily_hscore.index[k].count('Trump') == 1:
        tester.append(P_daily_hscore.index[k][0])

### The values are dropped in correspondence with the previous comments regarding cut-off of the tail end of the dictionary based methodology criterium.

In [ ]:
del catcher[-2:]
del tester[-2:]
df_B.drop(df_B.tail(2).index,inplace=True)
df_T.drop(df_T.tail(2).index,inplace=True)
df_B.index = catcher

In [ ]:
fig, ax = plt.subplots(figsize=(8,4), dpi=400)
ax.plot(df_B.index, df_B, ls = "--", alpha = 0.5, label='Biden')
ax.plot(df_T.index, df_T, ls = "--", alpha = 0.5, label='Trump')
plt.axvline(x=df_B.index[-1], color='green', alpha = 0.4, label='Election Day')
ax.set_title("Avg. daily happiness ratings \n r/politics submission comments")
ax.set_ylabel("Avg. happiness")
ax.xaxis.set_major_locator(plt.MaxNLocator(8))
plt.xticks(rotation = 25)
plt.ylim(5.1,7.1)
plt.grid(alpha=0.5)
ax.legend()
plt.show()

# Word Shifterator plots
We chose the election day as the day to remark upon, and compare it to the whole previous part of the dataset to investigate changes of sentiment over time. This amounts to a total of 33 days contained in the reference dictionary.

In [ ]:
election_day = P_daily_hscore.index[-5][0]
print('Election day is: {}'.format(election_day))
d_T = datetime.strptime(P_daily_hscore.index[-5][0], '%Y-%m-%d')
d_m = (d_T- dt.timedelta(days=33)).strftime('%Y-%m-%d') #string format

In [ ]:
l = documents_per_day.loc[election_day]
l_ref = np.concatenate(documents_per_day[(documents_per_day.index < election_day) & \
                                (documents_per_day.index > d_m)].values)

In [ ]:
p_l = dict([(item[0], item[1]/len(l)) for item in Counter(l).items()])
p_lref = dict([(item[0], item[1]/len(l)) for item in Counter(l_ref).items()])
sorted(p_lref.items(), key = lambda x:x[1], reverse=True)[:10]

In [ ]:
sorted(p_l.items(), key = lambda x:x[1], reverse=True)[:10]

So the main topic surrounds trump and people, however on the day of election voting seems to be more significant, understandably so.

In [ ]:
# [(token, diff(p_l, p_lref))]
# Do it for every word in the union of sets of words for corpora
d_p = dict([(token, p_l.get(token, 0) -  p_lref.get(token, 0)) \
               for token in set(p_l.keys()).union(set(p_lref.keys()))])
sorted(d_p.items(), key = lambda x:x[1], reverse=True)[:10]

# ^This doesn't tell us much

In [ ]:
labMT_dict = pd.Series(labMT["Happiness Score"].values, index=labMT.index).to_dict()
prep_hscore =  dict([(token, labMT_dict.get(token, np.nan)-5) for token in set(p_l.keys()).union(set(p_lref.keys()))])

d_phi = dict([(token, prep_hscore[token]*d_p[token]) for token in set(p_l.keys()).union(set(p_lref.keys()))\
                    if not np.isnan(prep_hscore[token])]) #Do it for everything that isn't NaN
# Absolute value sorting top 10
sorted(d_phi.items(), key = lambda x: np.abs(x[1]), reverse=True)[:10]

The $\delta\phi$ value identifies the extent a given set of words contribute to the difference in scores and how they do it. Either a word is used more often in one text over the other and its influence, positive or negative, is weighted in the relative frequency. 

In [ ]:
import shifterator as sft
senti_shift = sft.WeightedAvgShift(type2freq_1 = p_lref,
                                     type2freq_2 = p_l,
                                     type2score_1 = labMT_dict,
                                     reference_value = 5)
senti_shift.get_shift_graph(detailed = True,
                            system_names = ['ref', 'd'],
                           top_n=25)

Gone down on by 0.04 in comparison (neglible imo)